<a href="https://colab.research.google.com/github/Rtniewi/kcwiertniewicz-IDS/blob/main/GenAI/HW5/LSTM_3_Layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

//***Katrina Cwiertniewicz
//*** CSC 330
//11/22/2024
//Assignment 5: Text Generation Using LSTM on Project Gutenberg Training Data
####The purpose of this assignment is to develop an LSTM model that generates text. The goal is to produce coherent and stylistically relevant text based on prompts.

In [1]:
import numpy as np
import json
import re
import string

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, losses

## 0. Parameters <a name="parameters"></a>

In [2]:
VOCAB_SIZE = 20049
MAX_LEN = 500
EMBEDDING_DIM = 100
N_UNITS_LAYER_1 = 64
N_UNITS_LAYER_2 = 128
N_UNITS_LAYER_3 = 256
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 25

## 1. Load the data <a name="load"></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%pwd

'/content'

In [5]:
import requests
import os


# List of files for additional texts (e.g., different Edgar Allan Poe Works)
file_paths = [
  os.path.join('/content/drive/MyDrive/text/The_Tell_Tale_Heart.txt'),                        # The Tell Tale Heart
  os.path.join('/content/drive/MyDrive/text/The_Cask_of_Amontillado.txt'),                    # The Cask of Amontillado
  os.path.join('/content/drive/MyDrive/text/The_Raven.txt'),                                  # The Raven
  os.path.join('/content/drive/MyDrive/text/The_Masque.txt'),                                 # The Masque of the Red Death
  os.path.join('/content/drive/MyDrive/text/Annabel_Lee.txt'),                                # Annabel Lee
  os.path.join('/content/drive/MyDrive/text/Lenore.txt'),                                     # Lenore
  os.path.join('/content/drive/MyDrive/text/The_Bells.txt'),                                  # The Bells
  os.path.join('/content/drive/MyDrive/text/The_Black_Cat.txt'),                              # The Black Cat
  os.path.join('/content/drive/MyDrive/text/The_Fall_of_the_House_of_Usher.txt'),             # The Fall of the House of Usher
  os.path.join('/content/drive/MyDrive/text/The_Oval_Portrait.txt'),                          # The Oval Portrait
  os.path.join('/content/drive/MyDrive/text/The_Pit_and_the_Pendulum.txt'),                   # The Pit and the Pendulum
  os.path.join('/content/drive/MyDrive/text/The_Premature_Burial.txt'),                       # The Premature Burial
  os.path.join('/content/drive/MyDrive/text/The_Narrative_of_Arthur_Gordon.txt'),             # The Narrative of Arthur Gordon Pym of Nantucket
  os.path.join('/content/drive/MyDrive/text/Al_Aaraaf.txt'),                                  # Al Aaraaf
  os.path.join('/content/drive/MyDrive/text/Alone.txt'),                                      # Alone
  os.path.join('/content/drive/MyDrive/text/Eureka.txt'),                                     # Eureka
  os.path.join('/content/drive/MyDrive/text/The_Facts_in_the_Case_of_M._Valdemar.txt'),       # The Facts in the Case of M.Valdemar
  os.path.join('/content/drive/MyDrive/text/A_Descent_into_the_Maelstrom.txt'),               # A Descent into the Maelstrom
  os.path.join('/content/drive/MyDrive/text/William_Wilson.txt'),                             # William Wilson
  os.path.join('/content/drive/MyDrive/text/Berenice.txt'),                                   # Berenice
  os.path.join('/content/drive/MyDrive/text/The_Gold_Bug.txt'),                               # The Gold Bug
  os.path.join('/content/drive/MyDrive/text/The_Murders_of_Rue_Morgue.txt'),                  # The Murders in the Rue Morgue
  os.path.join('/content/drive/MyDrive/text/The_Mystery_of_Marie_Roget.txt'),                 # The Mystery of Marie Roget
  os.path.join('/content/drive/MyDrive/text/The_Purloined_Letter.txt'),                       # The Purloined Letter
  os.path.join('/content/drive/MyDrive/text/Von_Kempelen_and _his_Discovery.txt'),            # Von Kempelen and His Discovery
  os.path.join('/content/drive/MyDrive/text/Island_of_the_Fay.txt'),                          # Island of the Fay
  os.path.join('/content/drive/MyDrive/text/Mesmeric_Revelation.txt'),                        # Mesemeric Revelation
  os.path.join('/content/drive/MyDrive/text/Silence_A_Fable.txt'),                            # Silence a Fable
  os.path.join('/content/drive/MyDrive/text/MS._Found_in_a_Bottle.txt'),                      # MS. Found in a Bottle
  os.path.join('/content/drive/MyDrive/text/The_Thousand_and_Second_Tale_of_Scherezade.txt'), # The Thousand and Second Tale of Scherezade
  os.path.join('/content/drive/MyDrive/text/The_Unparalleled_Adventure.txt'),                 # The Unparalleled Adventure of One Hans Pfaall
  os.path.join('/content/drive/MyDrive/text/The_Assignation.txt'),                            # The Assignation
  os.path.join('/content/drive/MyDrive/text/The_Imp.txt'),                                    # The Imp of the Perverse
  os.path.join('/content/drive/MyDrive/text/The_Domain_of_Arnheim.txt'),                      # The Domain of Arnheim
  os.path.join('/content/drive/MyDrive/text/The_Assignation.txt'),                            # Landor's Cottage
  os.path.join('/content/drive/MyDrive/text/Morella.txt'),                                    # Morella
  os.path.join('/content/drive/MyDrive/text/Ligeia.txt'),                                     # Ligeia
  os.path.join('/content/drive/MyDrive/text/King_Pest.txt'),                                  # King Pest
  os.path.join('/content/drive/MyDrive/text/A_Tale_of_the_Ragged_Mountains.txt'),             # A Tale of the Ragged Mountains
  os.path.join('/content/drive/MyDrive/text/The_Spectacles.txt'),                             # The Spectacles
  os.path.join('/content/drive/MyDrive/text/Philosophy_of_Furniture.txt'),                    # The Philosophy of Furniture
  os.path.join('/content/drive/MyDrive/text/The_Devil_in_Belfry.txt'),                        # The Devil in the Belfry
  os.path.join('/content/drive/MyDrive/text/Bon_Bon.txt'),                                    # Bon-Bon
  os.path.join('/content/drive/MyDrive/text/Some_Words_with_a_Mummy.txt')                     # Some Words with a Mummy


]

# Initialize an empty string to hold all text
all_text = ""

# Download each text file and append to all_text
for file_path in file_paths:
  with open(file_path, 'r') as file:
    content = file.read()
    text = content
    all_text += text + "\n\n"  # Separate texts by newlines

# Save combined text to a single file
  with open('/content/combined_poe.txt', "w", encoding="utf-8") as file:
    file.write(all_text)


In [6]:
# Count the words of text
with open('/content/combined_poe.txt', "r", encoding="utf-8") as file:
  file_content = file.read()
  words = file_content.split()
  n_words = len(words)
print(f"{n_words} words loaded")

326500 words loaded


In [7]:
# Example Sentence of First Ten Words
example_sentence = words[:10]
print(f"Example Sentence: {example_sentence}")

Example Sentence: ['True!—nervous—very,', 'very', 'dreadfully', 'nervous', 'I', 'had', 'been', 'and', 'am;', 'but']


## 2. Tokenise the data

In [8]:
# Pad the punctuation, to treat them as separate 'words'
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


with open("combined_poe.txt", "r", encoding="utf-8") as file:
    text_data = [pad_punctuation(line) for line in file]

In [9]:
example_date = text_data[30]
print(example_date)

Ha ! —would a madman have been so wise as this ? And then , when my



In [10]:
# Convert to a Tensorflow Dataset
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

In [11]:
# Create a vectorisation layer
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [12]:
# Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

# Length of Vocabulary
print(f"Length of Vocabulary: {len(vocab)}")

Length of Vocabulary: 20049


In [13]:
# Display some token:word mappings
for i, word in enumerate(vocab[10:30]):
    print(f"{i}: {word}")

0: -
1: i
2: that
3: _
4: it
5: was
6: as
7: with
8: which
9: at
10: my
11: is
12: had
13: ;
14: we
15: this
16: for
17: by
18: not
19: be


In [14]:
# Display the same example converted to ints
example_tokenised = vectorize_layer(text_data)
print(example_tokenised.numpy())

[[  326    47 11929 ...     0     0     0]
 [   33   329    74 ...     0     0     0]
 [13630 18134 10712 ...     0     0     0]
 ...
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]]


## 3. Create the Training Set

In [15]:
# Create the training set of text and the same text shifted by one word
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)

## 4. Build the LSTM <a name="build"></a>

In [16]:
inputs = layers.Input(shape=(None,), dtype="int32")
x = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x = layers.LSTM(N_UNITS_LAYER_1, return_sequences=True)(x)
x = layers.LSTM(N_UNITS_LAYER_2, return_sequences=True)(x)
x = layers.LSTM(N_UNITS_LAYER_3, return_sequences=True)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
lstm = models.Model(inputs, outputs)
lstm.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, None, 100)           │       2,004,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, None, 64)            │          42,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, None, 128)           │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, None, 20049)         │       2,586,321 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,732,277 (18.05 MB)

 Trainable params: 4,732,277 (18.05 MB)

 Non-trainable params: 0 (0.00 B)

## 5. Train the LSTM <a name="train"></a>

In [17]:
loss_fn = losses.SparseCategoricalCrossentropy()
lstm.compile("adam", loss_fn)

In [18]:
# Create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }  # <1>

    def sample_from(self, probs, temperature):  # <2>
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]  # <3>
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0: # <4>
            x = np.array([start_tokens])
            y = self.model.predict(x, verbose=0)  # <5>
            sample_token, probs = self.sample_from(y[0][-1], temperature)  # <6>
            info.append({"prompt": start_prompt, "word_probs": probs})
            start_tokens.append(sample_token)  # <7>
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("Text:", max_tokens=1000, temperature=1.0)

In [19]:
# Tokenize starting prompt

text_generator = TextGenerator(vocab)

In [20]:
lstm.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[text_generator],
)

Epoch 1/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step - loss: 1.1330
generated text:
Text: awaking tanks vie dim superinduced the each very the de guy , in see thing one 

1035/1035 ━━━━━━━━━━━━━━━━━━━━ 215s 204ms/step - loss: 1.1323
Epoch 2/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step - loss: 0.1486
generated text:
Text: private the article ; and no result loathsome me - streamed on 

1035/1035 ━━━━━━━━━━━━━━━━━━━━ 211s 204ms/step - loss: 0.1486
Epoch 3/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step - loss: 0.1364
generated text:
Text: romanticist my secrecy . little on one ; while while no ever of the better up ’ we timber place . their seem of the my been —i lived in the writing winged or the realities upon my leaden , ? i overboard , and was while period evidence—the or . forty thing , and to 

1035/1035 ━━━━━━━━━━━━━━━━━━━━ 214s 206ms/step - loss: 0.1364
Epoch 4/25
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step - loss: 0.1333
generated text:
Text: . 

1035/1035 ━━━━━━━━━━━━━━━

## 6. Generate text using the LSTM

In [21]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        print(f"\nPROMPT: {i['prompt']}")
        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [22]:
# Prompt 1: From "The Raven"    Temperature=0.1
info = text_generator.generate(
    """Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more.""", max_tokens=500, temperature=0.1
)


generated text:
Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. . i could not have been told , i am not in the habit of my senses , alas ! i have been in vain . 



In [23]:
print_probs(info, vocab)


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more.
.:   	92.43%
,:   	3.11%
in:   	3.09%
;:   	0.7%
and:   	0.67%
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. .
the:   	42.2%
:   	22.16%
i:   	20.78%
it:   	14.76%
but:   	0.11%
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious v

In [42]:
# Prompt 1: From "The Raven"    Temperature=0.5
info = text_generator.generate(
    """Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more.""", max_tokens=1000, temperature=0.5
)


generated text:
Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. . it was a small floor of the most brilliant and unsteady character , upon the teeth of the lips , i had seriously awakened in my mind , and , in the manner of the mental nature is not not too much that i could not find the memory of my reflections , i had made no longer in my ears . 



In [43]:
print_probs(info, vocab)


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more.
.:   	30.71%
,:   	15.59%
in:   	15.57%
;:   	11.56%
and:   	11.45%
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. .
the:   	23.68%
:   	20.82%
i:   	20.55%
it:   	19.19%
but:   	7.21%
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and cur

In [26]:
# Prompt 1: From "The Raven"    Temperature=1.0
info = text_generator.generate(
    """Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more.""", max_tokens=1000, temperature=1.0
)


generated text:
Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. in the mansion ! and horror a madman fortunato died the character of the origin of its mind , is that of identity in the worm - like their physical common lay existence in revolution in proportion , and are at length irresistible , which , had been more and black , yet upon account was convinced it in perceiving the vulgar of the majestic temples in the library’s nearest of the sides , around them , was exceedingly unusual as i intended at length a devil , i saw a noose , the prey just to the interests , was not in the dying in identity , and in spite of the livid nature of the tortoise - lef back and lustreless alo

In [27]:
print_probs(info, vocab)


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more.
.:   	15.51%
,:   	11.05%
in:   	11.04%
;:   	9.51%
and:   	9.47%
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and curious volume of forgotten lore,
      While I nodded, nearly napping, suddenly there came a tapping,
      As of some one gently rapping, rapping at my chamber door.
      Tis some visitor, I muttered, tapping at my chamber door —
      Only this, and nothing more. in
the:   	31.43%
a:   	5.33%
their:   	5.1%
my:   	5.06%
its:   	4.83%
--------


PROMPT: Once upon a midnight dreary, while I pondered, weak and weary,
      Over many a quaint and cur

In [28]:
# Prompt 2: From "The Tell Tale Heart"  Temperature=0.1
info = text_generator.generate(
    """And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.""", max_tokens=1000, temperature=0.1
)


generated text:
And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage. . the teeth were carelessly in the most intense of the most intense and common beauty of the most intense and speculative object , the most hideous and piercing yet yet revelled of the most intense and speculative object , the most intense and speculative object , the most intense and speculative object , the most intense and speculative object , the most intense and speculative object , the most intense and speculative object , the hues of the most intense and rigid beauty of the library’s soul , the most intense and speculative object , the most intense and speculative object , the object of the ch

In [29]:
print_probs(info, vocab)


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.
.:   	99.61%
and:   	0.39%
,:   	0.0%
;:   	0.0%
—:   	0.0%
--------


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage. .
the:   	46.09%
:   	42.51%
i:   	11.4%
it:   	0.0%
this:   	0.0%
--------


PROMPT: And have I not told you that what you mistake for madness is but over acut

In [44]:
# Prompt 2: From "The Tell Tale Heart"  Temperature=0.5
info = text_generator.generate(
    """And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.""", max_tokens=1000, temperature=0.5
)


generated text:
And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage. and all the phantasma of the most ghastly and revelled of the most intense of the most intense and unsteady character , and from the first singular and yet most appalling and shadows of the most hideous and vivid thought it was not born in the armory — so that in the armory — but in a careless and a striking about yet fro , and i fell upon the bed of the chamber , the consuming shadows of the chamber was a gorge—although 



In [46]:
print_probs(info, vocab)


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.
.:   	64.53%
and:   	21.25%
,:   	8.1%
;:   	2.75%
—:   	2.08%
--------


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage. and
:   	50.74%
the:   	23.94%
a:   	14.38%
my:   	4.34%
—:   	1.37%
--------


PROMPT: And have I not told you that what you mistake for madness is but over

In [47]:
# Prompt 2: From "The Tell Tale Heart"  Temperature=1.0
info = text_generator.generate(
    """And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.""", max_tokens=1000, temperature=1.0
)


generated text:
And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage. . this was partially thus up . —“i ( even as both as the echo of the changed seemed is to have been on them . it was impossible secure the shadows of their evil frivolous monotonously the diameter of entire parts are obviously about , and indeed terror , however , it was , when , while night chamber sat , and in wild head sat , i whisperingly said , among his eyes and joy — now from to a careless station , and i sat upon my memory sentence around his single frame at my chamber — not a wide berenice ! — indeed the trunk in antique finger as it not , upon my eyes , and after a low , low effect , it was

In [48]:
print_probs(info, vocab)


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage.
.:   	37.09%
and:   	21.29%
,:   	13.14%
;:   	7.66%
—:   	6.66%
--------


PROMPT: And have I not told you that what you mistake for madness is but over acuteness of the senses? --
    now, I say, there came to my ears a low, dull, quick sound, such as a watch makes when enveloped in cotton.
     I knew that sound well, too. It was the beating of the old man's heart.
     It increased my fury, as the beating of a drum stimulates the soldier into courage. .
the:   	17.49%
:   	17.35%
i:   	15.21%
it:   	6.49%
this:   	3.66%
--------


PROMPT: And have I not told you that what you mistake for madness is but o

In [34]:
# Prompt 3: From the Mystery of Marie Roget   Temperature=0.1
info = text_generator.generate(
    """There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. """, max_tokens=1000, temperature=0.1
)


generated text:
There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them.  , and , in the case of the most intense and shrunken smile , in the contemplation of the most intense and shrunken draperies , in the contemplation of the most intense and shrunken smile , in the contemplation of the most intense and shrunken draperies , in the contemplation of the most intense and shrunken smile , in the contemplation of the most intense and shrunken smile , in the contemplation of the most intense and shrunken smile , in the contemplation of the most intense and shrunken smile , in the contemplation of the most intense and shrunken draperies , in the first instance , i have been in vain . i had not yet forgotten that i have been in vain . 



In [35]:
print_probs(info, vocab)


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. 
,:   	99.4%
in:   	0.6%
and:   	0.0%
as:   	0.0%
intermingled:   	0.0%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them.  ,
and:   	100.0%
in:   	0.0%
as:   	0.0%
with:   	0.0%
intermingled:   	0.0%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marve

In [49]:
# Prompt 3: From the Mystery of Marie Roget   Temperature=0.5
info = text_generator.generate(
    """There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. """, max_tokens=1000, temperature=0.5
)


generated text:
There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them.  , and , aggravated , in the grey of the most intense and shrunken frame , there , and , upon me , of the most intense and rigid sound , in the lucid period of the curtains of the existence of the mental nature is the most extraordinary and most intense and most intense nurse of the external world of the deceased , and of the intense and heavy meditation not yet no longer a thing in the fact is to be found in the earth’s identity of the most intense and common variations of the most appalling and rigid 



In [50]:
print_probs(info, vocab)


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. 
,:   	66.5%
in:   	23.95%
and:   	2.95%
as:   	2.78%
intermingled:   	1.04%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them.  ,
and:   	97.59%
in:   	0.76%
as:   	0.75%
with:   	0.4%
intermingled:   	0.18%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seeming

In [59]:
# Prompt 3: From the Mystery of Marie Roget   Temperature=1.0
info = text_generator.generate(
    """There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. """, max_tokens=1000, temperature=1.0
)


generated text:
There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them.  . the overflowing it fell upon out the straight light , and in full manner , is precisely but at defiance , one , is my natural frame of the most intense and spirit of that latter lurked ? ” 



In [60]:
print_probs(info, vocab)


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them. 
,:   	32.75%
in:   	19.65%
and:   	6.9%
as:   	6.7%
intermingled:   	4.1%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellous a character that, as
    mere coincidences, the intellect has been unable to receive them.  .
the:   	20.13%
i:   	10.33%
it:   	7.62%
:   	6.05%
there:   	4.49%
--------


PROMPT: There are few persons, even among the calmest thinkers, who have not
    occasionally been startled into a vague yet thrilling half-credence in the
    supernatural, by coincidences of so seemingly marvellou